# Validação Cruzada com Naive Bayes

## Atividade 6: Validacao_cruzada (k, dataset, ...) com Naive Bayes

1) Dividir dataset em k partes estratificadas
1.1) Determinar a quantidade de positivos, negativos, %pos e % neg de cada conjunto de folds (10 conjuntos)

2) Definir um array de valores de c (α-alpha), iniciando com 0.1 até 1.0 (10 valores)

3) Para Para cada valor de c (parâmetro (α-alpha) do classificador NB (NaiveBayes)):
    3.1) Para cada parte i, de 1 até k (sendo o i o indice do fold que será considerado para teste
    e os demais para o treinamento (j) sendo i ≠ j):
    
        3.1.1) Aplicar o classificador, com o c definido, sobre a parte i (predição) e calcular:
            - Acurácia (Acur_NB[c][i])
            - Revocação (Rev_BN[c][i])
            - Precisão (Prec_NB[c][i])
            
    3.2) Calcular a média dos Rev_NB[i], Prec_NB[i] e Acur_NB[i], isto é, será a média dos indicadores para o conjunto
    todo, que foi treinado e predito em partes.  

4) Calcular o intervalo de confiança das médias de Rev_NB[c], Prec_NB[c] e Acur_NB[c]

5) Analisar os resultados obtidos, observando os resultados para cada c (alpha) aplicado.

6) Usar a Acurácia como balizador de escolha e identificar o parâmetro c (ótimo) de maior acurácia

7) Retreinar o conjunto todo com o c ótimo

8) Calcular o intervalo de confiança de Rev_NB[cotimo], Prec_NB[cotimo] e Acur_NB[cotimo]

9) Aplicação dos itens 1 ao 8. As etapas de 1 a 8, acima, deverão ser aplicadas para:
    - Todo o dataset (conjunto inteiros de características/componentes)
    - O dataset resultante da aplicação do PCA (com somente as características calculadas retornadas pelo PCA)
    - Os dataset´s resultantes dos dois selecionadores, com as características indicadas pelos selecionadores 


10) Apresentação:
- Informações gerais do dataset (do que se trata a classificação binária, quantas instâncias, proporção de cada classe)

- Métodos:

    -- Tratamento realizado no dataset para uso com Naive Bayes (NB).
    
    -- Rebalanceamento utilizado (qual método)
    
    -- Pesquisa de missing values. Tratamento dos casos e método usado para correção.
    
    -- Aplicação de normalizalição. Qual.
    
    -- Implementação do NB. Bibliotecas utilizadas, adaptações.
    
    -- Implementou algum método, isto é, não utilizou frameworks prontos
    
    -- Como realizou a calibração. Parâmetros utilizados.

- Resultados

    -- Tabela Comparativa e gráficos

- Discussão
    
    -- O que se esperava do treinamento e predições, variações no uso do alpha e no uso de validação cruzada

    -- O conjunto de dados do dataset é adequado para este treinador (Naive Bayes)
    
    -- Discrepâncias e percepções dos resultados (% de acertos)esperado e obtido
    
    -- etc

In [41]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

import statsmodels.stats.api as sms

from mpl_toolkits.mplot3d import Axes3D

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import random
random.seed(42)

## 1) Leitura da Base

In [42]:
# Bases a serem utilizadas - 
# 'df_final_20200510.csv' - 'df_kbest_fclassif_20200510.csv' - 'df_pca_features_importances_20200510.csv'
# 'df_random_forest_importances_20200510.csv' - 'df_RFE_20200510.csv' - 'df_SBS_20200510.csv' - 'df_SFS_20200510.csv'

df = pd.read_csv('df_RFE_20200510.csv')

#print("Número de linhas e colunas:", df.shape, '\n')
#print('\n', df.info(), '\n')

#df.head()

## 2) Preparação do dataset

In [43]:
# Preparação e separação da base para as demais atividades
dfx = df.copy()
dfx.reset_index(inplace=True, drop=True)

# Armazenando as classes
classes = dfx['TARGET'].value_counts()
attributes = list(df.columns)

# Guardando a coluna Target em Y e removendo a coluna TARGET da base principal
Y = dfx.loc[:, 'TARGET']
dfx = dfx.drop('TARGET', axis=1)

# Guardando a quantidade de linhas e colunas após a separação da coluna TARGET
nrow, ncol = dfx.shape

# Armazenando as colunas (atributos)
attributes = list(dfx.columns)

# Transformando para numpy
X = dfx.to_numpy()

# Normalizando
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

#dfx.head()

## 3) Partição do dataset em k folds
<br> Como o dataset completo possui 217 linhas e 329 colunas, optamos em utilizar o SVD no lugar do PCA com 
dois componentes, já que no SVD não é possível trabalhar com todos os componentes.</br>

In [44]:
############################################################
#### Função de separação de uma base em k sub-conjuntos ####
#### Recebe:                                            ####
#### -- data: dataframe dos atributos                   ####
#### -- target: array com a coluna alvo                 ####
#### -- k: quantidade desejada de separação             ####
####                                                    ####
#### Retorna:                                           ####
#### -- Os índices a serem considerados em cada parte   ####
############################################################
def splitFolds(data, target, k=10):
    
    # Contadores para apresentação
    ldata = len(data)              # Quantidade de linhas da base
    numel = int(ldata / k)         # Quantidade de amostras por fold
        
    uclass = target.value_counts() # Classes e suas quantidades
    uclass.sort_index(inplace=True)
    nclass = uclass.index          # Classes
    qclass = uclass.values         # Quantidade de cada classe
    
    # Junção das classes e suas quantidades para um dicionário
    zclass = zip(nclass, qclass)   
    dclass = dict(zclass)          
    
    # Separação dos conjuntos
    partesK = []    # Conterá todos os conjuntos k de índices, cada um proporcional a cada classe
    for i in range(k):
        pk = []
        if (i < 7):
            nelem = numel + 1
        else:
            nelem = numel
            
        ntot = nelem
        
        for nc, qc in dclass.items():
            # Captura de todos os índices da coluna target
            masc = target == nc
            idclass = list(target[masc].index)
        
            # Montagem dos k subgrupos com a mesma proporção da classe
            propclass = int(round(nelem * qc / ldata))
            if (ntot > propclass):
                ntot -= propclass
            else:
                propclass = ntot
            
            rs = random.sample(idclass, propclass)
            pk = pk + rs
            
        partesK.append(pk)
        
    return (partesK)          


In [45]:
# Separação do dataset
k = 10
partesK = splitFolds(X, Y, k)

# Estatísticas da separação
qtdDS = len(X)
vc = Y.value_counts()
qtdT0 = vc.values[1]
qtdT2 = vc.values[0]
perT0 = round(qtdT0/qtdDS * 100, 2)
perT2 = round(qtdT2/qtdDS * 100, 2)

# Dicionário onde serão guardadas as informações do Dataset e suas execuções
# chave ds - descrição geral de quantidades de classes e suas porcentagens
dicSet = {}
dicSet['ds'] = {}
dicSet['ds']['qtdC0'] = qtdT0
dicSet['ds']['qtdC2'] = qtdT2
dicSet['ds']['perC0'] = perT0
dicSet['ds']['perC2'] = perT2

qtd0 = 0
qtd2 = 0
per0 = 0.0
per2 = 0.0

# chave folds - descrição de cada separação
dicSet['ds']['fs'] = {}
dicSet['ds']['fs']['k'] = k

# Separação dos folds
for f in range(len(partesK)):
    dx = Y[partesK[f]]
    vc = dx.value_counts()
    qtd0 = vc.values[1]
    qtd2 = vc.values[0]
    per0 = round(qtd0/(qtd0 + qtd2) * 100, 2)
    per2 = round(qtd2/(qtd0 + qtd2) * 100, 2)
    
    dicSet['ds']['fs'][f+1] = {}
    dicSet['ds']['fs'][f+1]['qtdC0'] = qtd0
    dicSet['ds']['fs'][f+1]['qtdC2'] = qtd2
    dicSet['ds']['fs'][f+1]['perC0'] = per0
    dicSet['ds']['fs'][f+1]['perC2'] = per2


In [46]:
dicSet

{'ds': {'qtdC0': 101,
  'qtdC2': 116,
  'perC0': 46.54,
  'perC2': 53.46,
  'fs': {'k': 10,
   1: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   2: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   3: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   4: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   5: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   6: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   7: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   8: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38},
   9: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38},
   10: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38}}}}

## 4) Aplicação do Naive Bayes - Bernouilli

In [49]:
# Dicionário onde serão guardadas as informações do Dataset e suas execuções
# - chave ds - descrição geral de quantidades de classes e suas porcentagens
# - chave fs - descrição de cada separação - o fold de teste e para cada fold a quantidade e porcentagem
# - chave NB - treinamento e aplicação do Naive Bayes, com:
# -- Dicionário com as respostas dos treinamentos:
# -- Por Parâmetro alpha
# --- O Fold de teste considerado para predição (os demais são para treinamento)
# ---- Quantidade de classes de predição
# ---- Acurácia, Revocação, Precisão

qtdRodadas = len(partesK)

# Hiperparâmetro para treinamento - de 0.1 a 1.0
alpha = list(np.arange(0.1, 1.1, 0.1))

# Dicionário do Naive Bayes
NB = {}  

# Laço de repetição para todos alphas
for al in alpha:    
    # Cria uma entrada de chave no dicionário para cada alpha
    NB[al] = {}
    
    # Laço de repetição para quantidade de folds
    for ft in range(qtdRodadas):
        # Cria uma entrada no dicionário (abaixo de alpha) para cada fold escolhido como teste
        NB[al][ft] = {'classes':[], 'Acur':[], 'Rev':[], 'Prec':[]}
    
        # Montagem dos índices das partes de Teste e Treino
        iTreino = []
        iTeste = []
        for i, p in enumerate(partesK):
            if (i == ft):
                # Fold das partesK que será utilizada para o teste
                iTeste = p
                continue
        
            # Montando os demais folds para Treinamento
            for a in p:
                iTreino.append(a)
        
        # Organiza os indices a serem considerados
        iTreino.sort()
        iTeste.sort()
    
        # Aplicando os índices para separar o dataset em conjunto de treinamento e teste
        x_train, x_test = X[iTreino], X[iTeste]
        y_train, y_test = Y[iTreino], Y[iTeste]
        
        # Treinando e aplicando o modelo NB
        modelGNB = BernoulliNB(alpha = al)
        modelGNB.fit(x_train, y_train)
        y_pred = modelGNB.predict(x_test)
    
        # Cria um elemento no dicionário (abaixo de alpha e fold, guardando as quantidade de amostras por classe)
        v, c = np.unique(y_pred, return_counts=True)
        NB[al][ft]['classes'] = list(c)   
            
        # Calculando a matriz de confusão
        mat = list(confusion_matrix(y_test, y_pred))
        vp = mat[0][0]
        vn = mat[0][1]
        fp = mat[1][0]
        fn = mat[1][1]
        # Acurácia = VP + VN)/n
        NB[al][ft]['Acur'] = (vp + vn) / len(y_pred)
        # Revocação = VP / (VP + FN))
        NB[al][ft]['Rev'] = vp / (vp + fn)
        # Precisão (VP / (VP + FP))
        NB[al][ft]['Prec'] = vp / (vp + fp)
                
# Dicionário avalia, utilizado para os treinamentos e predições será incorporado ao dicionário dicSet
dicSet['ds']['NB'] = NB


In [50]:
dicSet

{'ds': {'qtdC0': 101,
  'qtdC2': 116,
  'perC0': 46.54,
  'perC2': 53.46,
  'fs': {'k': 10,
   1: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   2: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   3: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   4: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   5: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   6: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   7: {'qtdC0': 10, 'qtdC2': 12, 'perC0': 45.45, 'perC2': 54.55},
   8: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38},
   9: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38},
   10: {'qtdC0': 10, 'qtdC2': 11, 'perC0': 47.62, 'perC2': 52.38}},
  'NB': {0.1: {0: {'classes': [10, 12],
     'Acur': 0.45454545454545453,
     'Rev': 0.4375,
     'Prec': 0.7},
    1: {'classes': [13, 9],
     'Acur': 0.45454545454545453,
     'Rev': 0.5384615384615384,
     'Prec': 0.5384615384615384},
    2

In [62]:
# Calculando as médias dos kfolds por parâmetro alpha
nb = dicSet['ds']['NB']

# Quantidade de kfolds
n = dicSet['ds']['fs']['k']

# Guardando num dicionário de médias
dicMedia = {}

# Por cada alpha - ka
for ka, va in nb.items():
    dicMedia[ka] = {}
    
    # Por cada fold acumula a acurácia, precisão e revocação e determina a média de cada
    ttclasses = [0.00, 0.00]
    tacu = 0.00
    trev = 0.00
    tpre = 0.00
    lacu = []
    lrev = []
    lpre = []
    for kf, vf in va.items():
        ttclasses[0] += vf['classes'][0]
        ttclasses[1] += vf['classes'][1]
        tacu += vf['Acur']
        trev += vf['Rev']
        tpre += vf['Prec']
        
        lacu.append(vf['Acur'])
        lrev.append(vf['Rev'])
        lpre.append(vf['Prec'])

    # Calculando a média
    dicMedia[ka]['QtClasses'] = ttclasses
    dicMedia[ka]['M_Acu'] = tacu / n
    dicMedia[ka]['M_Rev'] = trev / n
    dicMedia[ka]['M_Pre'] = tpre / n

    # Calculo do Intervalo de Confiança
    dicMedia[ka]['I_Acu'] = sms.DescrStatsW(lacu).tconfint_mean(alpha=0.05)
    dicMedia[ka]['I_Rev'] = sms.DescrStatsW(lrev).tconfint_mean(alpha=0.05)
    dicMedia[ka]['I_Pre'] = sms.DescrStatsW(lpre).tconfint_mean(alpha=0.05)

In [63]:
dicMedia

{0.1: {'QtClasses': [115.0, 102.0],
  'M_Acu': 0.461038961038961,
  'M_Rev': 0.49465538847117785,
  'M_Pre': 0.6543123543123543,
  'I_Acu': (0.453559519168725, 0.46851840290919705),
  'I_Rev': (0.45745717458010376, 0.531853602362252),
  'I_Pre': (0.5843104173103604, 0.7243142913143481)},
 0.2: {'QtClasses': [115.0, 102.0],
  'M_Acu': 0.461038961038961,
  'M_Rev': 0.49465538847117785,
  'M_Pre': 0.6543123543123543,
  'I_Acu': (0.453559519168725, 0.46851840290919705),
  'I_Rev': (0.45745717458010376, 0.531853602362252),
  'I_Pre': (0.5843104173103604, 0.7243142913143481)},
 0.30000000000000004: {'QtClasses': [116.0, 101.0],
  'M_Acu': 0.461038961038961,
  'M_Rev': 0.4991425679583574,
  'M_Pre': 0.6504662004662005,
  'I_Acu': (0.453559519168725, 0.46851840290919705),
  'I_Rev': (0.45778707389216, 0.540498062024555),
  'I_Pre': (0.5764210730464657, 0.7245113278859351)},
 0.4: {'QtClasses': [117.0, 100.0],
  'M_Acu': 0.461038961038961,
  'M_Rev': 0.5023476961634856,
  'M_Pre': 0.65379953379

In [68]:
# Gravando uma planilha em excel
saida = pd.DataFrame(dicMedia)
saida.to_csv('Arq_NB_01_20200513.csv') #, index=False)